In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objs as go

from scipy.optimize import least_squares # least squares non linear

from scipy.integrate import quad
from scipy import real

In [2]:
df = pd.read_csv('data1.csv')

In [3]:
df

,Spot,Maturity,Strike,Interest_rate,Mid,Bid,Ask
0,328.29,0.175342,275,0.000554,56.90,55.5,58.3
1,328.29,0.175342,300,0.000554,36.30,35.0,37.6
2,328.29,0.175342,325,0.000554,19.60,19.3,19.9
3,328.29,0.175342,350,0.000554,9.45,9.2,9.7
4,328.29,0.175342,375,0.000554,4.30,4.1,4.5
5,328.29,0.424658,275,0.000659,63.20,61.7,64.7
6,328.29,0.424658,300,0.000659,44.90,44.4,45.4
7,328.29,0.424658,325,0.000659,30.55,30.2,30.9
8,328.29,0.424658,350,0.000659,20.05,19.7,20.4
9,328.29,0.424658,375,0.000659,12.50,12.2,12.8


### ON IMPLEMENTE LA LEAST SQUARE NON LINEAR

In [4]:
# FONCTION CHARACTERISTIQUE ln(ST) DE HESTON

def chfun_heston(s0, v0, vbar, a, vvol, r, rho, t, w):
    """
    Heston characteristic function.
    Inputs:
    s0: stock price
    v0: initial volatility (v0^2 initial variance)
    vbar: long-term variance mean
    a: variance mean-reversion speed
    vvol: volatility of the variance process
    r : risk-free rate
    rho: correlation between the Weiner processes for the stock price and its variance
    t: time
    w: points at which to evaluate the function
    Output:
    y: Characteristic function of log (St) in the Heston model
    """
    # Interim calculations
    alpha = -w*w/2 - 1j*w/2
    beta = a - rho*vvol*1j*w
    gamma = vvol*vvol/2
    h = np.sqrt(beta*beta - 4*alpha*gamma)
    rplus = (beta + h)/(vvol*vvol)
    rminus = (beta - h)/(vvol*vvol)
    g = rminus / rplus

    # Required inputs for the characteristic function
    C = a * (rminus * t - (2 / (vvol**2)) * np.log((1 - g * np.exp(-h*t))/(1 - g)))
    D = rminus * (1 - np.exp(-h * t))/(1 - g * np.exp(-h*t))

    # Characteristic function evaluated at points w
    y = np.exp(C*vbar + D*v0 + 1j*w*np.log(s0*np.exp(r*t)))
    return y


In [5]:
# POUR AVOIR LA FORMULE DU CALL

def call_heston_cf(s0, v0, vbar, a, vvol, r, rho, t, k, chfun_heston):
    """
    Heston call value using characteristic functions.
    Inputs:
    s0: stock price
    v0: initial volatility (v0^2 initial variance)
    vbar: long-term variance mean
    a: variance mean-reversion speed
    vvol: volatility of the variance process
    r: risk-free rate
    rho: correlation between the Weiner processes of the stock price and its variance
    t: time to maturity
    k: option strike
    chfun_heston: Heston characteristic function
    Output:
    y: Heston call value
    """
    # Inner integral 1
    def int1(w, s0, v0, vbar, a, vvol, r, rho, t, k):
        return np.real(np.exp(-1j * w * np.log(k)) * chfun_heston(s0, v0, vbar, a, vvol, r, rho, t, w - 1j) /
                       (1j * w * chfun_heston(s0, v0, vbar, a, vvol, r, rho, t, -1j)))
    
    int1_result, _ = quad(lambda w: int1(w, s0, v0, vbar, a, vvol, r, rho, t, k), 0, 100)
    pi1 = int1_result / np.pi + 0.5

    # Inner integral 2
    def int2(w, s0, v0, vbar, a, vvol, r, rho, t, k):
        return np.real(np.exp(-1j * w * np.log(k)) * chfun_heston(s0, v0, vbar, a, vvol, r, rho, t, w) / (1j * w))
    
    int2_result, _ = quad(lambda w: int2(w, s0, v0, vbar, a, vvol, r, rho, t, k), 0, 100)
    pi2 = int2_result / np.pi + 0.5

    # Calculate call value
    y = s0 * pi1 - np.exp(-r * t) * k * pi2
    return y

In [10]:
def mse(params):
    v0, vbar, a, eta, rho = params

    # call_heston_cf(s0, v0, vbar, a, vvol, r, rho, t, k, chfun_heston)

    s=0
    for i in range(df.shape[0]):
        s += (call_heston_cf(df['Spot'].iloc[i], v0, vbar, a, eta, df['Interest_rate'].iloc[i], rho, df['Maturity'].iloc[i], df['Strike'].iloc[i], chfun_heston) - df['Mid'].iloc[i])**2
    return s/df.shape[0]    

In [11]:
x0 = np.array([0.5, 0.5, 1, 1, -0.5]) 
lb = np.array([0, 0, 0, 0, -1])
ub = np.array([1, 1, 20, 5, 1])

res = least_squares(mse, x0, bounds=(lb, ub))

In [12]:
res

 active_mask: array([0, 0, 0, 0, 0])
        cost: 0.11970994377833397
         fun: array([0.48930552])
        grad: array([-0.07137806, -0.14124793, -0.05597005,  0.12093429, -0.01971363])
         jac: array([[-0.14587625, -0.28867023, -0.1143867 ,  0.24715497, -0.040289  ]])
     message: 'The maximum number of function evaluations is exceeded.'
        nfev: 500
        njev: 445
  optimality: 0.9380932449175625
      status: 0
     success: False
           x: array([ 0.11761414,  0.27654127,  3.23936952,  3.77379053, -0.15046367])